In [7]:
from typing import NamedTuple, List, Tuple
from collections import Counter
from tensorflow import keras
from keras.layers import *
import keras.backend as K
from keras.models import Model
from collections import defaultdict
import numpy as np
from sklearn.model_selection import train_test_split
import urllib.request

In [19]:
def read_basket_data_from_url(url):
    dataset = []
    with urllib.request.urlopen(url) as data:
        for line in data:
            line = line.decode("utf-8")
            products = [int(p)-1 for p in line.split(',')]
            if len(products) > 1:
                dataset.append(products)
    return dataset

def build_vocab(dataset: List[List[int]]):
    counter = Counter()
    for basket in dataset:
        counter.update(basket)
    return list(counter.keys()), counter

In [20]:
def to_size(data: List[int], size: int):
    if len(data) > size:
        return np.random.choice(data, size=size, replace=False)
    else:
        return np.random.choice(data, size=size, replace=True)

class BasketData(NamedTuple):
    dataset: Tuple[np.ndarray, np.ndarray]
    vocab: List[int]
    counter: Counter
    vocab_size: int
    max_basket_length: int
    
    @staticmethod
    def build_from_url(url: str):
        dataset = read_basket_data_from_url(url)
        print(f"Read {len(dataset)} baskets from {url}")
        
        vocab, counter = build_vocab(dataset)
        print(f"Number of distinct products {len(vocab)}")
        
        max_basket_length = max(len(b) for b in dataset)
        print(f"Max basket size {max_basket_length}")
        
        dataset = BasketData.build_input_and_labels(dataset, max_basket_length)
        
        print(f"Done building dataset")
        return BasketData(dataset, vocab, counter, len(vocab), max_basket_length)
    
    @staticmethod
    def build_input_and_labels(baskets: List[List[int]], max_length: int) -> Tuple[np.ndarray, np.ndarray]:
        inputs = []
        labels = []
        for basket in baskets:
            input_basket = basket[:-1]
            label_product = basket[-1]

            inputs.append(to_size(input_basket, max_length))
            labels.append(label_product)

        inputs = np.array(inputs)
        labels = np.array(labels)
        return inputs, labels

In [21]:
data_url = "https://www.dropbox.com/s/hkwnwlut4mb5yyb/1_100_100_100_apparel_regs.csv?dl=1"
basket_data = BasketData.build_from_url(data_url)

Read 8102 baskets from https://www.dropbox.com/s/hkwnwlut4mb5yyb/1_100_100_100_apparel_regs.csv?dl=1
Number of distinct products 100
Max basket size 21
Done building dataset


In [22]:
def BasketCNN(max_sequence_length, vocab_size, embedding_dim=100, num_filters=16, dropout_rate=0.25):
    """
    Input:
        - max_sequence_length: maximum length of baskets
        - vocab_size: number of distinct products
        - embedding_layer: embedding layer of Keras created by model type and static flags
        - dropout_rate: dropout rate for flattened pooled outputs
    Returns:
        - model: Model class created with specified inputs
    """        
    x_input = Input(shape=(max_sequence_length,), dtype='int32')

    embedding_layer = Embedding(input_dim=vocab_size,
                                output_dim=embedding_dim,
                                embeddings_initializer='uniform')

    x = embedding_layer(x_input)

    kernel_sizes = [3, 5, 7]
    pooled = []

    for kernel in kernel_sizes:

        conv = Conv1D(filters=num_filters,
                      kernel_size=kernel,
                      padding='valid',
                      strides=1,
                      kernel_initializer='he_uniform',
                      activation='relu')(x)
        
        pool = MaxPooling1D(pool_size=max_sequence_length - kernel + 1)(conv)

        pooled.append(pool)

    merged = Concatenate(axis=-1)(pooled)

    flatten = Flatten()(merged)

    drop = Dropout(rate=dropout_rate)(flatten)
    
    x_output = Dense(vocab_size, kernel_initializer='he_uniform', activation='softmax')(drop)

    return Model(inputs=x_input, outputs=x_output)

In [24]:
model = BasketCNN(basket_data.max_basket_length, basket_data.vocab_size, dropout_rate=0.25)
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 21)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 21, 100)      10000       input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 19, 16)       4816        embedding_1[0][0]                
__________________________________________________________________________________________________
conv1d_2 (Conv1D)    

In [26]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='Adam',
              metrics=['sparse_categorical_accuracy', 'sparse_top_k_categorical_accuracy'])

In [27]:
X, y = basket_data.dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=128, epochs=10, verbose=1)

Instructions for updating:
Use tf.cast instead.
Train on 6481 samples, validate on 1621 samples
Epoch 1/10
6481/6481 [==============================] - 1s 207us/step - loss: 4.4140 - sparse_categorical_accuracy: 0.1906 - sparse_top_k_categorical_accuracy: 0.3012 - val_loss: 3.9913 - val_sparse_categorical_accuracy: 0.2579 - val_sparse_top_k_categorical_accuracy: 0.4442
Epoch 2/10
6481/6481 [==============================] - 1s 135us/step - loss: 3.5815 - sparse_categorical_accuracy: 0.2600 - sparse_top_k_categorical_accuracy: 0.4536 - val_loss: 3.4416 - val_sparse_categorical_accuracy: 0.2708 - val_sparse_top_k_categorical_accuracy: 0.4547
Epoch 3/10
6481/6481 [==============================] - 1s 137us/step - loss: 3.3891 - sparse_categorical_accuracy: 0.2631 - sparse_top_k_categorical_accuracy: 0.4794 - val_loss: 3.3738 - val_sparse_categorical_accuracy: 0.2782 - val_sparse_top_k_categorical_accuracy: 0.4682
Epoch 4/10
6481/6481 [==============================] - 1s 138us/step - loss

In [28]:
most_common_product, max_count = basket_data.counter.most_common(1)[0]

In [29]:
naive_labels = np.zeros_like(y_test, dtype=np.float32)
naive_labels = most_common_product

In [30]:
(naive_labels == y_test).mean()

0.23442319555829735